# Agent Setup

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool
from agents.run import RunConfig
from pydantic import BaseModel


load_dotenv()


GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    raise ValueError('Gemini API Key is not valid.')


external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.0-flash',
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

## Tool raising error

In [2]:
@function_tool
def divide(a: float, b: float) -> float:
    '''
    Performs division between two numbers

    Args:
        a: dividend
        b: diviser
    '''

    print('divide tool called')
    # return a / b
    raise ValueError('Not Good values')


agent = Agent(
    name='Supportive Agent',
    instructions='Give helpful and concise answers',
    model=model,
    tools=[divide]
)

res = await Runner.run(agent, 'What is 8/2?', run_config=config)
print(res.final_output)

divide tool called
I am sorry, I cannot perform this calculation at this time. Please try again later.



In [27]:
@function_tool
def divide(a: float, b: float) -> float:
    '''
    Performs division between two numbers

    Args:
        a: dividend
        b: diviser
    '''
    print('divide tool called')
    try:
        raise ZeroDivisionError("manual error for test")
        return a / b
    except Exception as e:
        raise ValueError(f'Error during division: {e}')

agent2 = Agent(
    name='Supportive Agent',
    instructions='Must Use tool to answer',
    model=model,
    tools=[divide]
)

res = await Runner.run(agent2, 'What is 8/4', run_config=config)
print(res.final_output)

divide tool called
I am unable to calculate this at the moment. Please try again later.



## Hooks

In [21]:
from agents.lifecycle import RunHooks, AgentHooks

# Define a sample tool
@function_tool
def echo_tool(message: str) -> str:
    """Echoes the input message."""
    return f"ECHO: {message}"

# Run-level hooks for all agent runs
class MyRunHooks(RunHooks):
    async def on_agent_start(self, context, agent):
        print(f"[RUN HOOK] Starting agent: {agent.name}")

    async def on_tool_start(self, context, agent, tool: Tool):
        print(f"[RUN HOOK] Agent '{agent.name}' is invoking tool: {tool.name}")

    async def on_tool_end(self, context, agent, tool, result: str):
        print(f"[RUN HOOK] Tool '{tool.name}' returned: {result}")

    async def on_agent_end(self, context, agent, output):
        print(f"[RUN HOOK] Agent '{agent.name}' finished with output: {output}")

# Agent-level hooks for one specific agent
class MyAgentHooks(AgentHooks):
    async def on_start(self, context, agent):
        print(f"[AGENT HOOK] {agent.name} started")

    async def on_tool_start(self, context, agent, tool: Tool):
        print(f"[AGENT HOOK] {agent.name} is about to call {tool.name}")

    async def on_tool_end(self, context, agent, tool, result: str):
        print(f"[AGENT HOOK] {agent.name} got tool result: {result}")

    async def on_end(self, context, agent, output):
        print(f"[AGENT HOOK] {agent.name} ended with: {output}")

# Create agent and attach hooks
agent = Agent(
    name="EchoAgent",
    instructions="Echo back your input using the echo_tool.",
    tools=[echo_tool],
    model=model,
    hooks=MyAgentHooks()
)
agent.hooks = MyAgentHooks()

# Create runner with run-level hooks
result = await Runner.run(agent, "Hello, world!", run_config=config, hooks=MyRunHooks())
print("Final answer:", result.final_output)

[RUN HOOK] Starting agent: EchoAgent
[AGENT HOOK] EchoAgent started
[RUN HOOK] Agent 'EchoAgent' is invoking tool: echo_tool
[AGENT HOOK] EchoAgent is about to call echo_tool
[RUN HOOK] Tool 'echo_tool' returned: ECHO: Hello, world!
[AGENT HOOK] EchoAgent got tool result: ECHO: Hello, world!
[RUN HOOK] Agent 'EchoAgent' finished with output: Hello, world!

[AGENT HOOK] EchoAgent ended with: Hello, world!

Final answer: Hello, world!



## Streaming with Gemini

In [28]:
from openai.types.responses import ResponseTextDeltaEvent


streaming_agent = Agent(
    name="Joker",
    instructions="You are a joke telling assistant.",
    model=model
)

result = Runner.run_streamed(streaming_agent, input="Please tell me 5 jokes.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Alright, buckle up, buttercup! Here are 5 jokes ready to launch:

1.  Why don't scientists trust atoms? Because they make up everything!

2.  Parallel lines have so much in common. It's a shame they'll never meet.

3.  Why did the scarecrow win an award? Because he was outstanding in his field!

4.  I used to hate facial hair... but then it grew on me.

5.  What concert costs just 45 cents? 50 Cent featuring Nickelback!

Hope at least one of those tickled your funny bone!  Let me know if you want another round!
